In [1]:
# -*- coding: utf-8
# VOLT DATA LAB (https://www.voltdata.info/)
# Reinaldo Chaves (@paidatocandeira)
# Mostra quais projetos de lei tiveram aprovação e as falhas de transparência do sistema 
#

In [2]:
import json
import xmltodict
import pandas as pd
import numpy as np
from datetime import date

# Tramitações

# O sistema de dados abertos da Alesp tem um arquivo enorme de histórico de tramitações

https://www.al.sp.gov.br/dados-abertos/recurso/101

# Mas esse arquivo tem erros de formatação nos anos mais antigos e não abre

Há uma alternativa, um arquivo menor e bem formatado, mas ele cobre só até 09/2017

Atual

https://www.al.sp.gov.br/dados-abertos/recurso/221

In [4]:
with open("dados/tramitacao/documento_andamento_atual_set_2018.xml", 'r') as f:
    xmlString = f.read()

In [5]:
jsonString = json.dumps(xmltodict.parse(xmlString), indent=4)

In [6]:
with open("output.json", 'w') as f:
    f.write(jsonString)

In [7]:
arquivo_json = open('output.json', 'r')
dados_json = json.load(arquivo_json)

In [8]:
#dados_json['documentos_autores']['DocumentoAutor']

In [9]:
lista = dados_json['documentos_andamentos']['DocumentoAndamento']

In [10]:
df_andamentos = pd.DataFrame(lista, columns = ['Descricao', 
                                               'Data', 
                                               'IdComissao', 
                                               'IdDocumento', 
                                               'IdEtapa', 
                                               'IdTpAndamento', 
                                               'NmEtapa', 
                                               'NrOrdem', 
                                               'TpAndamento'])

In [11]:
df_andamentos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 518496 entries, 0 to 518495
Data columns (total 9 columns):
Descricao        512734 non-null object
Data             518496 non-null object
IdComissao       518496 non-null object
IdDocumento      518496 non-null object
IdEtapa          518496 non-null object
IdTpAndamento    518496 non-null object
NmEtapa          518496 non-null object
NrOrdem          518496 non-null object
TpAndamento      518496 non-null object
dtypes: object(9)
memory usage: 35.6+ MB


In [13]:
df_andamentos.reset_index().head()

,index,Descricao,Data,IdComissao,IdDocumento,IdEtapa,IdTpAndamento,NmEtapa,NrOrdem,TpAndamento
0,0,Arquivado pelo Setor de Arquivo na caixa: 13.0...,2017-09-15T00:00:00-03:00,0,166,17,32,Arquivo,0,E
1,1,Arquivado pelo Setor de Arquivo na caixa: 13.0...,2017-09-15T00:00:00-03:00,0,167,17,32,Arquivo,0,E
2,2,Arquivado pelo Setor de Arquivo na caixa: 13.0...,2017-09-15T00:00:00-03:00,0,171,17,32,Arquivo,0,E
3,3,Arquivado pelo Setor de Arquivo na caixa: 13.0...,2017-09-15T00:00:00-03:00,0,176,17,32,Arquivo,0,E
4,4,Arquivado pelo Setor de Arquivo na caixa: 13.0...,2017-09-15T00:00:00-03:00,0,181,17,32,Arquivo,0,E


In [14]:
#a = df_andamentos['Descricao'].value_counts()

# Filtra só as linhas que têm alguma aprovação de projeto de lei

In [17]:
mask = df_andamentos['Descricao'].str.contains("Aprovado o Projeto", na=False)
aprovado = df_andamentos[mask]

In [18]:
aprovado.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 271 entries, 30103 to 492104
Data columns (total 9 columns):
Descricao        271 non-null object
Data             271 non-null object
IdComissao       271 non-null object
IdDocumento      271 non-null object
IdEtapa          271 non-null object
IdTpAndamento    271 non-null object
NmEtapa          271 non-null object
NrOrdem          271 non-null object
TpAndamento      271 non-null object
dtypes: object(9)
memory usage: 21.2+ KB


In [30]:
aprovado.to_excel('resultados/projetos_aprovados.xlsx',sheet_name='Sheet1')

# Carrega a base de palavras-chave/projetos criada no notebook alesp_projetos

In [19]:
palavras_chave_dep = pd.read_excel("resultados/projetos_deputados_palavraschave.xlsx", sheet_name='Sheet1', converters={'IdDocumento': lambda x: str(x), 
                                                                                                                         'IdPalavra': lambda x: str(x), 
                                                                                                                                 'IdAutor': lambda x: str(x), 
                                                                                                                         'CodOriginalidade': lambda x: str(x), 
                                                                                                                         'IdNatureza': lambda x: str(x), 
                                                                                                                         'NroLegislativo': lambda x: str(x),
                                                                                                                         'codigo_unico': lambda x: str(x),
                                                                                                                         'sequencial': lambda x: str(x),
                                                                                                                         'cpf': lambda x: str(x)
                                                                                                                         })

In [20]:
palavras_chave_dep.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31778 entries, 0 to 31777
Data columns (total 23 columns):
IdAutor             31778 non-null object
IdDocumento         31778 non-null object
NomeAutor           31778 non-null object
codigo_unico        31778 non-null object
nome_deputado       31778 non-null object
uf                  31778 non-null object
nome_completo       31778 non-null object
sequencial          31778 non-null object
cpf                 31778 non-null object
nome_urna           31778 non-null object
partido_eleicao     31778 non-null object
situacao            31778 non-null object
AnoLegislativo      31778 non-null int64
CodOriginalidade    24339 non-null object
DtEntradaSistema    31778 non-null datetime64[ns]
DtPublicacao        31778 non-null datetime64[ns]
Ementa              31778 non-null object
IdNatureza          31778 non-null object
NroLegislativo      31778 non-null object
chave_unica         31778 non-null object
IdPalavra           31778 non-null o

In [21]:
projetos_aprovados = pd.merge(palavras_chave_dep, aprovado, left_on='IdDocumento', right_on='IdDocumento')

In [22]:
projetos_aprovados.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6325 entries, 0 to 6324
Data columns (total 31 columns):
IdAutor             6325 non-null object
IdDocumento         6325 non-null object
NomeAutor           6325 non-null object
codigo_unico        6325 non-null object
nome_deputado       6325 non-null object
uf                  6325 non-null object
nome_completo       6325 non-null object
sequencial          6325 non-null object
cpf                 6325 non-null object
nome_urna           6325 non-null object
partido_eleicao     6325 non-null object
situacao            6325 non-null object
AnoLegislativo      6325 non-null int64
CodOriginalidade    878 non-null object
DtEntradaSistema    6325 non-null datetime64[ns]
DtPublicacao        6325 non-null datetime64[ns]
Ementa              6325 non-null object
IdNatureza          6325 non-null object
NroLegislativo      6325 non-null object
chave_unica         6325 non-null object
IdPalavra           6325 non-null object
Palavra           

In [23]:
len(projetos_aprovados['NomeAutor'].value_counts())

95

In [24]:
a = len(projetos_aprovados['IdDocumento'].value_counts())
a

199

Deputados com projetos aprovados

In [26]:
projetos_aprovados.groupby('NomeAutor')['IdDocumento'].nunique().sort_values(ascending=False)

NomeAutor
Sebastião Santos                     10
Pedro Tobias                          8
Carlão Pignatari                      7
Edson Giriboni                        7
Roberto Engler                        7
Caio França                           7
Márcio Camargo                        7
Welson Gasparini                      6
Afonso Lobato                         6
Itamar Borges                         6
Fernando Cury                         5
Chico Sardelli                        5
Luiz Carlos Gondim                    5
Marcos Damasio                        5
Mauro Bragato                         5
Paulo Correa Jr                       5
Maria Lúcia Amary                     5
Roque Barbiere                        5
Roberto Massafera                     4
Rita Passos                           4
Luiz Fernando T. Ferreira             4
Wellington Moura                      4
Célia Leão                            4
Davi Zaia                             4
João Paulo Rillo              

Palavras-chave dos projetos aprovados

In [27]:
projetos_aprovados.groupby('Palavra')['IdDocumento'].nunique().sort_values(ascending=False)

Palavra
DENOMINAÇÃO                               68
TURISMO                                   28
LAZER                                     24
DESENVOLVIMENTO TURÍSTICO                 24
DESENVOLVIMENTO ECONÔMICO E SOCIAL        24
ESTÂNCIA TURÍSTICA                        24
VIADUTO                                   18
RODOVIA                                   14
ESCOLA                                    13
MUNICÍPIO DE INTERESSE TURÍSTICO          11
PASSARELA                                  8
PONTE                                      7
PROIBIÇÃO                                  5
DISPOSITIVO DE ACESSO                      5
INTERESSE TURÍSTICO                        5
MOGI DAS CRUZES (MUNICÍPIO)                4
BARRETOS (MUNICÍPIO)                       4
TREVO                                      4
CONSCIENTIZAÇÃO                            4
COMPLEXO VIÁRIO                            3
ESTADO DE SÃO PAULO                        3
SÃO JOÃO DA BOA VISTA (MUNICÍPIO)          3
CR